In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


In [2]:

# Carregue os DataFrames origem_ID e text do CSV


caminho_arquivo_1= 'cod_material_servico.txt'
origem_ID = pd.read_csv(caminho_arquivo_1,sep=";" )

caminho_arquivo_2= 'text.txt'
text = pd.read_csv(caminho_arquivo_2,sep="," )


In [ ]:
# Função para encontrar a correspondência fuzzy e retornar a correspondência com o maior score
def find_best_match(row):
    matches = process.extractOne(row['classif'], text['dsc_item'], scorer=fuzz.ratio)
    return pd.Series([row['classif'], matches[0]])

# Aplicar a função de correspondência fuzzy e criar o novo DataFrame
matches_df = origem_ID.apply(find_best_match, axis=1)
matches_df.columns = ['classif', 'melhor_classif']

# Salvar o DataFrame resultante em um novo CSV
matches_df.to_csv('resultado.csv', index=False)

# Exibir as primeiras linhas do DataFrame resultante
print(matches_df.head())




In [ ]:
#Limpando text.txt

text['dsc_item'] = text['dsc_item'].astype(str)
df["dsc_item"] = df["dsc_item"].apply(lambda x: x.lower()) #converte todas as letras para minúsculo 
df["dsc_item"] = df["dsc_item"].apply(lambda x: re.sub('|,|\.|/|$|\(|\)|-|\+|:|•', '', x)) #remove números e caracteres especiais 
df["dsc_item"] = df["dsc_item"].apply(lambda x: unidecode(x)) #remove acentos 
stemmer = nltk.stem.RSLPStemmer() #converte as palavras para seu radical 
df["dsc_item"] = df["dsc_item"].apply(lambda x: stemmer.stem(x))

# Função para encontrar a correspondência fuzzy e retornar a correspondência com o maior score
def find_best_match(row):
    matches = process.extractOne(row['classif'], text['dsc_item'], scorer=fuzz.ratio)
    return pd.Series([row['classif'], matches[0], matches[1]])

# Aplicar a função de correspondência fuzzy e criar um novo DataFrame com os resultados
resultados = origem_ID.apply(find_best_match, axis=1)
resultados.columns = ['classif_origem', 'melhor_dst_item', 'score']

# Filtre apenas as linhas com um score acima de um determinado limite (opcional)
score_limite = 80  # Você pode ajustar esse limite conforme necessário
resultados_filtrados = resultados[resultados['score'] >= score_limite]

# Salvar o DataFrame resultante em um novo CSV
resultados_filtrados.to_csv('resultado.csv', index=False)

# Exibir as primeiras linhas do DataFrame resultante
print(resultados_filtrados.head())
